In [1]:
# import os.path

# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

# # If modifying these scopes, delete the file token.json.
# SCOPES = ["https://www.googleapis.com/auth/youtube.readonly","https://www.googleapis.com/auth/youtube.force-ssl"]


# creds = None
# # The file token.json stores the user's access and refresh tokens, and is
# # created automatically when the authorization flow completes for the first
# # time.
# if os.path.exists("token.json"):
#   creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# # If there are no (valid) credentials available, let the user log in.
# if not creds or not creds.valid:
#   if creds and creds.expired and creds.refresh_token:
#     creds.refresh(Request())
#   else:
#     flow = InstalledAppFlow.from_client_secrets_file(
#         "credentials.json", SCOPES
#     )
#     creds = flow.run_local_server(port=0)
#   # Save the credentials for the next run
#   with open("token.json", "w") as token:
#     token.write(creds.to_json())

# try:
#   youtube = build("youtube", "v3", credentials=creds)

#       # Taking input from the user and slicing for video id
#   video_id = input('Enter Youtube Video URL: ')[-11:]
#   print("video id: " + video_id)
  
#   # Getting the channelId of the video uploader
#   video_response = youtube.videos().list(
#       part='snippet',
#       id=video_id
#   ).execute()
  
#   print(video_response)
#   # Splitting the response for channelID
#   video_snippet = video_response['items'][0]['snippet']
#   uploader_channel_id = video_snippet['channelId']
#   print("channel id: " + uploader_channel_id)
  
# except HttpError as err:
#   print(err)


In [ ]:
from googleapiclient.discovery import build
import re

API_KEY = ''  # Put in your API Key

youtube = build('youtube', 'v3', developerKey=API_KEY)  # initializing Youtube API

# Improved input handling for video ID
def extract_video_id(url):
    match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    return match.group(1) if match else None

video_url = input('Enter YouTube Video URL: ')
video_id = extract_video_id(video_url)

if not video_id:
    print("Invalid YouTube URL")
else:
    print("video id: " + video_id)

    # Getting the channelId of the video uploader
    try:
        video_response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()

        print("Video Response:", video_response)  # Debugging step

        if 'items' in video_response and video_response['items']:
            video_snippet = video_response['items'][0]['snippet']
            uploader_channel_id = video_snippet.get('channelId', 'No channelId found')
            print("channel id: " + uploader_channel_id)

            # Fetch comments
            print("Fetching Comments...")
            comments = []
            nextPageToken = None
            while len(comments) < 600:
                request = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    maxResults=100,  # You can fetch up to 100 comments per request
                    pageToken=nextPageToken
                )
                response = request.execute()
                for item in response['items']:
                    comment = item['snippet']['topLevelComment']['snippet']
                    # Check if the comment is not from the video uploader
                    if comment['authorChannelId']['value'] != uploader_channel_id:
                        comments.append(comment['textDisplay'])
                nextPageToken = response.get('nextPageToken')

                if not nextPageToken:
                    break
            # Print the first 5 comments
            print(comments[:5])

        else:
            print("No video found for the provided ID.")
    except Exception as e:
        print("An error occurred:", e)


# Channel to CsV

In [3]:
from googleapiclient.discovery import build
import pandas as pd
from emoji import demojize
import os

# Set up the YouTube API client
api_key = os.getenv('YOUTUBE_API_KEY')  # Make sure to set this environment variable
youtube = build('youtube', 'v3', developerKey=api_key)

def get_video_stats(video_id):
    try:
        response = youtube.videos().list(
            part='statistics',
            id=video_id
        ).execute()
        
        if 'items' in response and response['items']:
            stats = response['items'][0]['statistics']
            return {
                'likes': int(stats.get('likeCount', 0)),
                'dislikes': int(stats.get('dislikeCount', 0))  # Note: YouTube API no longer provides dislike counts
            }
        else:
            return {'likes': 0, 'dislikes': 0}
    except Exception as e:
        print(f"Error fetching stats for video {video_id}: {str(e)}")
        return {'likes': 0, 'dislikes': 0}

# Load the CSV file
df = pd.read_csv('MrBeast_video_comments.csv', encoding='utf-8')

# Display the first few rows to see the structure
print(df.head())

# Check if we have a 'Video ID' column
if 'Video ID' not in df.columns:
    print("Warning: 'Video ID' column not found. We need this to fetch likes and dislikes.")
else:
    # Get unique video IDs
    unique_video_ids = df['Video ID'].unique()
    print(f"Number of unique videos: {len(unique_video_ids)}")
    print("First few video IDs:", unique_video_ids[:5])

print("\
Dataframe columns:", df.columns.tolist())

In [ ]:
import time
start_time = time.time()
# Replace with the actual YouTube channel ID
channel_id = f'UCX6OQ3DkcsbYNE6H8uQQuVA'
fetch_videos_and_comments(channel_id, max_videos=100, max_comments_per_video=100)
end_time = time.time()
execution_time = start_time - end_time
print("Execution time:",execution_time)

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# is_cuda = torch.cuda.is_available()

# # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
# if is_cuda:
#     device = torch.device("cuda")
#     print("GPU is available")
# else:
#     device = torch.device("cpu")
#     print("GPU not available, CPU used")

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Load the CSV file
df = pd.read_csv('MrBeast_video_comments.csv', encoding='utf-8')

# Display the first few rows and basic information about the dataset
print(df.head())
print(df.info())

# Preprocess the data
# Use the 'Comment' column as our input and 'Video Title' as our target
X = df['Comment'].values
y = df['Video Title'].values

  Channel Name                              Video Title  \
0      MrBeast  Running With Bigger And Bigger Lunchlys   
1      MrBeast  Running With Bigger And Bigger Lunchlys   
2      MrBeast  Running With Bigger And Bigger Lunchlys   
3      MrBeast  Running With Bigger And Bigger Lunchlys   
4      MrBeast  Running With Bigger And Bigger Lunchlys   

                                  Comment  
0                              I love you  
1  Please give me one iPhone😢😢😢😢!🥺🥺🥺🥺🥺🥺🥺🥺  
2       ❤❤😂😂🎉😢😮😮😅😊😊❤❤❤😂😂😂🎉🎉😢😢😢😮😮😅😅😊😊❤❤😂😂😢  
3        It&#39;s a lot it&#39;s all good  
4         Can you buy a car for my dad ??  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Channel Name  9800 non-null   object
 1   Video Title   9800 non-null   object
 2   Comment       9800 non-null   object
dtypes: object(3)
memory usage: 229.8+ KB
None


In [20]:
import torch
from emoji import demojize


# Update the preprocessing function to include emoji conversion
def preprocess_text_with_emojis(text):
    # Convert emojis to text
    
    emoji_pattern = re.compile(r':([a-zA-Z_]+):')
    unique_emojis = set(emoji_pattern.findall(text))
    
    # Reconstruct the text with only one instance of each emoji
    for emote in unique_emojis:
        text = re.sub(f'(:{emote}:)+', f':{emote}:', text)

    # Convert to lowercase
    text = text.lower()
    # Remove special characters, numbers, and extra whitespace
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Reprocess the comments with the updated function
df['processed_comment_with_emojis'] = df['Comment'].apply(preprocess_text_with_emojis)

print("\
First 5 processed comments with emojis:")

print(df['processed_comment_with_emojis'].head())

def preprocess_comment(comment):
    # Replace emojis with their meanings
    return demojize(comment, language='en')

# Preprocess the comments
df['Preprocessed_Comment'] = df['Comment'].apply(preprocess_comment)

# Display the first few rows of the preprocessed data
print(df[['Comment', 'Preprocessed_Comment']].head())

# Convert preprocessed comments to PyTorch tensors
comment_tensors = [torch.tensor([ord(c) for c in comment]) for comment in df['Preprocessed_Comment']]

# Display the first tensor
print("\
First comment tensor:")
print(comment_tensors[0])

# Print the shape of the first tensor
print("\
Shape of the first tensor:", comment_tensors[0].shape)

# Print the total number of tensors
print("\
Total number of tensors:", len(comment_tensors))

First 5 processed comments with emojis:
0                      i love you
1       please give me one iphone
2                                
3          its a lot its all good
4    can you buy a car for my dad
Name: processed_comment_with_emojis, dtype: object
                                  Comment  \
0                              I love you   
1  Please give me one iPhone😢😢😢😢!🥺🥺🥺🥺🥺🥺🥺🥺   
2       ❤❤😂😂🎉😢😮😮😅😊😊❤❤❤😂😂😂🎉🎉😢😢😢😮😮😅😅😊😊❤❤😂😂😢   
3        It&#39;s a lot it&#39;s all good   
4         Can you buy a car for my dad ??   

                                Preprocessed_Comment  
0                                         I love you  
1  Please give me one iPhone:crying_face::crying_...  
2  :red_heart::red_heart::face_with_tears_of_joy:...  
3                   It&#39;s a lot it&#39;s all good  
4                    Can you buy a car for my dad ??  
First comment tensor:
tensor([ 73,  32, 108, 111, 118, 101,  32, 121, 111, 117])
Shape of the first tensor: torch.Size([10])
Total number 